In [1]:
import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths

from src.utils import load_ms_data

In [2]:
df = load_ms_data("all")

In [3]:
# Extract all the entries that contain "nebenwirkung" in the text
df_se = df.filter(lambda e: "nebenwirkung" in e["text"].lower())

In [7]:
print(df_se["train"]["text"][0])

1. Schubförmige Multiple Sklerose (RRMS) (EM 1994, ED 1995), EDSS 6.5
DD sekundär progredienter Verlauf mit aufgesetzten Schüben
Aktuell: klinisch: stabil, radiologisch: geringe Aktivität, Progression: ja (nach Lublin et al. 2013)
Klinisch: Visusminderung 0.4 rechts, linke 0.8, spastische Paraparese der Beine links > rechts; Hypästhesie bds sub Th10/11, sowie Halsbereich rechts; Patient bis 120m mobil am Rollator mobil
Verlauf:
INDENT 1994: Retrobulbärneuritis rechts
INDENT 08/2000: Schub mit sensomotorischer Paraparese der Beine (= aktenanamnestisch 7. Schub)
INDENT 10/2005: Schub mit INO rechts
INDENT Rollstuhl seit 2005 (noch knapp 600m mit Rollator)
INDENT 01/2010: Schub mit Retrobulbärneuritis rechts und Doppelbildern nach rechts, Hypästhesie linken Unterarms
INDENT 12/2014: Schub mit nukleärer Fazialisparese links mit Hörminderung links mit Prednison 20mg 6d und Ausschleichen bei V.a. periphere Fazialisparese behandelt.
INDENT 28.01.2015 Schub mit vertikalen Doppelbilder, Visusmi

In [8]:
import pandas as pd
df = pd.read_csv(paths.DATA_PATH_PREPROCESSED/"text-finetune/kisim_diagnoses_combined.csv")

In [9]:
# Filter out all the entries that contain "nebenwirkung" in the text
df_se = df[df["text"].str.contains("nebenwirkung", case=False, na=False)]

In [15]:
for text in df_se["text"]:
    text = text.splitlines()
    text = [t for t in text if "nebenwirkung" in t.lower()]
    print(text)
    print("-"*50)

['INDENT 12/2014 - 08/2015 Tecfidera, sistiert bei gastrointestinalen Nebenwirkungen']
--------------------------------------------------
['INDENT 11/97 - 02/05: Rebif (Stopp wegen Nebenwirkungen)', 'INDENT 08/08 - 10/09: Rebif 3x44 ug (erneuter Beginn; wg. Nebenwirkungen abgesetzt) ']
--------------------------------------------------
['Nicht näher bezeichnete unerwünschte Nebenwirkung eines Arzneimittels oder einer Droge']
--------------------------------------------------
['INDENT 11/2014 - 08/2017 Tecfidera, keine relevanten Nebenwirkungen']
--------------------------------------------------
['INDENT St.n. Therapie mit Tecfidera 02/2019 (Abbruch bei Gastrointestinalen Nebenwirkungen)']
--------------------------------------------------
['INDENT St.n. symptomatischer Therapie mit Fampyra (insgesamt 2 Therapieversuche, gestoppt bei Wirkungslosogkeit und i.R. der kardialen Vorbefunden), Therapieversuch mit Modasomil/Remeron (keine Besserung), Venlafaxin (bei Nebenwirkungen gestoppt), 

In [12]:
len(df_se)

232

- Medications:
    - Will add extra string field for LLM to specify anything else (like daily, weekly etc.) and if no schema is detected put -99 for everything.
    - CoT prompting could be done but is more for mathematical reasoning tasks
        - Medications chain could be useful for schema (as this probably most difficult)
            1. Extract medication name, unit and amount (seems to be working pretty well, is directly in text).
            2. Is there a schema like 1-0-0 or similar in the text related to the {medication, amount, unit} (yes, no)
                1. If no put -99 for all days.
                    1. There is no schema but is there a specification for how the medication has to be taken (daily, weekly etc.)? (Put in “extra” field)
                2. If yes: 
                    1. extract the schema
                    2. The {schema} you extracted relates to morning-noon-evening-night. Thus the dose you have to take of this medication is: 
                    3. Does the schema change over time (yes, no) (put in “extra” field)
- Side-effects CoT:
    1. Extract all the possible medications from text
    2. Iterate over medications: did this medication cause side effects (nebenwirkungen in german) (yes, no)
        1. If yes: is the side effect mentioned? (yes, no)